In [1]:
import numpy as np
from matplotlib import pyplot as plt

### import best weights obtained after training

In [2]:
theta1 = np.load('mnist/best2_theta_1.npy')
theta2 = np.load('mnist/best2_theta_2.npy')
theta1.shape,theta2.shape

((30, 785), (10, 31))

## A neural network with 2 hidden layers
# first hidden-layer : 30 cells

In [3]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [4]:
def predict(tempX):
    a1 = np.insert(tempX,0,np.ones(1),axis=0)
    z2 = np.dot(a1,theta1.T)
    a2 = sigmoid(z2)
    a2 = np.insert(a2,0,np.ones(1),axis=0)
    z3 = np.dot(a2,theta2.T)
    a3 = sigmoid(z3)
#     return a1, z2, a2, z3, h
    return np.asarray(a3)

## Now the paint app

In [5]:
from tkinter import *
import tkinter.ttk as ttk
import tkinter as tk
from tkinter import colorchooser
from tkinter import filedialog
from PIL import Image, ImageDraw, ImageGrab, ImageTk
import PIL
from tkinter import messagebox
import win32gui
import matplotlib.image as mpimg

In [6]:
%matplotlib qt

In [9]:
root = Tk()
root.title('Program.com Paint Program')
#root.geometry ('800*800')

bg_color = "black"
brush_color = "white"


def paint(e):
    
    brush_width = "{:.0f}".format(my_slider.get())
    brush_type2 = brush_type.get()
    
    # starting position
    x1 = e.x -1
    y1 = e.y -1
    # ending position
    
    x2 = e.x +1
    y2 = e.y +1
    
    #Draw on canvas
    
    my_canvas.create_line(x1, y1, x2, y2, fill=brush_color, width=brush_width, capstyle=brush_type2, smooth=True)


#Change size of brush
def change_brush_size(thing):
    slider_label.config(text='%0.0f' % float(my_slider.get()))

#Change brush color
def change_brush_color():
    global brush_color
#     brush_color = "black"
    brush_color = "white"
    brush_color = colorchooser.askcolor(color=brush_color)[1]

#Clear Screen
def clear_screen():
    my_canvas.delete(ALL)
    my_canvas.config(bg="black")
    my_canvas.delete("all")


# #Save Image
# def save_as_png():
#     result = filedialog.asksaveasfilename(initialdir='c:/paint/images/', filetypes=(("png files", "*.png"), ("all files", "*.*")))
    
#     if result.endswith('.png'):
#         pass
#     else:
#         result = result + '.png'

#     if result:
#         HWND = my_canvas.winfo_id() # get the handle of the canvas
#         rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
#         im = ImageGrab.grab(rect).save(result)
# #         x=root.winfo_rootx()+my_canvas.winfo_x()+10
# #         y=root.winfo_rooty()+my_canvas.winfo_y()+50
# #         x1=x+my_canvas.winfo_width()-30
# #         y1=y+my_canvas.winfo_height()-30
# #         ImageGrab.grab().crop((x,y,x1,y1)).save(result)
        
#         #Pop Up success message box
#         messagebox.showinfo("Image Saved", "Your image has been saved!")

#     result_label = Label(root, text=result)
#     result_label.pack(pady=20)
    
    
# predicting digit drawn on canvas

def predict_digit_drawn():

    HWND = my_canvas.winfo_id() # get the handle of the canvas
    rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas

    img = ImageGrab.grab(rect)
    img = img.convert('L')
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = np.asarray(img)
    print(img.shape)
    print('---------------------------')
    #reshaping to support our model input and normalizing
    img = img.reshape(-1)
#     print('*'*50)
#     print("resized and flatten now",img.shape)
#     print('*'*50)
#     print(img[:50])
#     print('*'*50)    
    img = img/255.0
#     print(img[:50])
#     print('*'*50)
    img = (img - img.mean())
    img = img/img.std()
#     print(img[:50])
#     print('*'*50)
    probab = predict(img)
    for i in range(10):
        print(i,"-> {:.8f}".format(probab[i]))
    print('-'*50)
    print("you've drawn ",np.argmax(probab))
#     digit_drawn = im.convert('L') 

#     digit_drawn = digit_drawn.resize((28,28))

#     arr = np.asarray(digit_drawn)

#     tempX = arr.reshape(-1)
# #     normalize
#     tempX = tempX - tempX.mean()
#     tempX = tempX/tempX.std()
# #  ----------------
#     h = feed_forward(tempX)

    
#     print("probabilities : ")
#     for i in range(10):
#         print(i,"-> {:.8f}".format(h[i]),end="\n")
#     print('*'*50)
    x = np.arange(10)
    plt.bar(x, height=h)
    plt.xticks(x, [str(i) for i in range(10)])
#     plt.imshow(img.reshape((28,28)),cmap="Greys_r")
    plt.suptitle("predicted digit is : "+str(np.argmax(probab)))
#     plt.xlabel("digit")
    
#     plt.ylabel("probability")
    
#szie of our canvas
w = 400
h = 500

my_canvas = Canvas(root, width=w, height=h, bg="black")
my_canvas.pack(pady=20)


my_canvas.bind('<B1-Motion>', paint)

#Brush options frame
brush_options_frame = Frame(root)
brush_options_frame.pack(pady=20)

#Brush size
brush_size_frame = LabelFrame(brush_options_frame, text="Brush size")
brush_size_frame.grid(row=0, column=0, padx=50)

#Brush slider
my_slider = ttk.Scale(brush_size_frame, from_=1, to=100, command=change_brush_size, orient=VERTICAL, value=10)
my_slider.pack(pady=10, padx=10)

#Brush slider label
slider_label = Label(brush_size_frame, text=my_slider.get())
slider_label.pack(pady=5)


brush_type = StringVar()
brush_type.set("round")

#Change colors
change_colors_frame = LabelFrame(brush_options_frame, text="Change colors")
change_colors_frame.grid(row=0, column=2)

#Change brush color button
brush_color_button = Button(change_colors_frame, text="Brush Color", command=change_brush_color)
brush_color_button.pack(pady=10, padx=10)


#Program options frame
options_frame = LabelFrame(brush_options_frame, text="Program Options")
options_frame.grid(row=0,column=3,padx=50)

#Clear screen button
clear_button = Button(options_frame, text="Clear Screen", command=clear_screen)
clear_button.pack(padx=10, pady=10) 

# #Save image
# save_image_button = Button(options_frame, text="Save to PNG", command=save_as_png)
# save_image_button.pack(pady=10, padx=10)

#Predict digit
predict_button = Button(options_frame, text="Predicting digit", command=predict_digit_drawn)
predict_button.pack(pady=10, padx=10)

In [10]:
root.mainloop()

(28, 28)
---------------------------
0 -> 0.00000000
1 -> 0.00000000
2 -> 0.00000000
3 -> 0.00002189
4 -> 0.00000000
5 -> 0.00017252
6 -> 0.00000000
7 -> 0.00000000
8 -> 0.88996050
9 -> 0.00000895
--------------------------------------------------
you've drawn  8
(28, 28)
---------------------------
0 -> 0.00000000
1 -> 0.00000001
2 -> 0.00000000
3 -> 0.00042455
4 -> 0.00004892
5 -> 0.00083061
6 -> 0.00000159
7 -> 0.00000000
8 -> 0.00010521
9 -> 0.00902217
--------------------------------------------------
you've drawn  9
